<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/Optimized%20SWAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Autokeras library


# Import required libraries
import timeit
import numpy as np
import tensorflow as tf
# from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Dense, concatenate, Dropout, Flatten, Activation
from keras import backend as K
from keras.utils import get_custom_objects
from keras.utils import  to_categorical, plot_model
from tensorflow.keras.optimizers import Adam
from keras.datasets import mnist


# Set random seed
np.random.seed(110)
# Defining new activation functions
def define_activation_functions():
    """
    Define custom activation functions and update the custom objects dictionary.
    """
    # Define activation functions and their respective names
    activation_functions = [
        ('X_1', lambda x: K.pow(x, 1)),
        ('X_2', lambda x: K.pow(x, 2) / 2),
        ('X_2_', lambda x: K.pow(x, 2) / 24),
        ('X_2__', lambda x: K.pow(x, 2) / 720),
        ('X_2___', lambda x: K.pow(x, 2) / 40320),
        ('X_3', lambda x: K.pow(x, 3) / 6),
        ('X_4', lambda x: K.pow(x, 4) / 24),
        ('X_5', lambda x: K.pow(x, 5) / 120),
        ('X_6', lambda x: K.pow(x, 6) / 720),
    ]

    # Update the custom objects dictionary with the defined activation functions
    for name, function in activation_functions:
        get_custom_objects().update({name: Activation(function)})

# Define and register the custom activation functions
define_activation_functions()

def create_optimized_model(input_dim,  hidden_dim , output_dim , metrics_ ):
    """
    Create an optimized SWAG model with a custom architecture.

    :param input_dim: int, dimension of the input data
    :param output_dim: int, dimension of the output data
    :param hidden_dim: int, hidden layer dimension, default is 50
    :return: Model, a compiled Keras model
    """

    # Define input layer
    input_layer = Input(shape=(input_dim,))

    # First layer with custom activations
    layer_1_x1 = Dense(hidden_dim, activation='X_1')(input_layer)
    layer_1_x2 = Dense(hidden_dim, activation='X_2')(input_layer)
    concat_first_layer = concatenate([layer_1_x1, layer_1_x2])

    # Second layer with custom activations
    layer_x3_x4 = Dense(hidden_dim, activation='X_2_')(concat_first_layer)

    # Third layer with custom activations
    layer_x5_x6 = Dense(hidden_dim, activation='X_2__')(layer_x3_x4)



    # Concatenate all layers
    concat_second_layer = concatenate([layer_1_x1, layer_1_x2, concat_first_layer,
                                       layer_x3_x4, layer_x5_x6])

    # Output layer for the concatenated layers
    output_first_layer = Dense(hidden_dim, activation='linear')(concat_second_layer)

    # Final output layer
    output_layer = Dense(output_dim, activation='linear')(output_first_layer)

    # Create and compile the model
    model = Model(input_layer, output_layer)
    # Define the optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=metrics_)
    # model.compile(loss='mean_squared_error', optimizer='adam')

    return model


# optimized_model = create_optimized_model(784,784, 10)

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = (x_train + 10) / 300.0, (x_test + 10) / 300.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create and train the model
optimized_model = create_optimized_model(784, 500, 10 , 'accuracy')
batch_size = 100
epochs = 2
start = timeit.default_timer()
history = optimized_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/2
600/600 [==============================] - 13s 5ms/step - loss: 0.0410 - accuracy: 0.8389 - val_loss: 0.0254 - val_accuracy: 0.9099
Epoch 2/2
600/600 [==============================] - 3s 4ms/step - loss: 0.0204 - accuracy: 0.9371 - val_loss: 0.0196 - val_accuracy: 0.9353
Training time: 16.397827482000004 seconds


In [11]:
# Load and preprocess Iris data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)


num_features = X_train.shape[1]
num_classes = y_train.shape[1]



# Create and train the model
model = create_optimized_model(num_features, 10, num_classes, 'accuracy')
batch_size = 10
epochs = 10
start = timeit.default_timer()
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

Epoch 1/10
12/12 [==============================] - 3s 20ms/step - loss: 0.6485 - accuracy: 0.5167 - val_loss: 0.4542 - val_accuracy: 0.5667
Epoch 2/10
12/12 [==============================] - 0s 7ms/step - loss: 0.3776 - accuracy: 0.6667 - val_loss: 0.2697 - val_accuracy: 0.6333
Epoch 3/10
12/12 [==============================] - 0s 7ms/step - loss: 0.2349 - accuracy: 0.7333 - val_loss: 0.1786 - val_accuracy: 0.7333
Epoch 4/10
12/12 [==============================] - 0s 6ms/step - loss: 0.1655 - accuracy: 0.7750 - val_loss: 0.1354 - val_accuracy: 0.8667
Epoch 5/10
12/12 [==============================] - 0s 6ms/step - loss: 0.1250 - accuracy: 0.8417 - val_loss: 0.1113 - val_accuracy: 0.9000
Epoch 6/10
12/12 [==============================] - 0s 7ms/step - loss: 0.1068 - accuracy: 0.8417 - val_loss: 0.0989 - val_accuracy: 0.9000
Epoch 7/10
12/12 [==============================] - 0s 6ms/step - loss: 0.0961 - accuracy: 0.8750 - val_loss: 0.0915 - val_accuracy: 0.9333
Epoch 8/10
12/12 [=

In [17]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
digits = load_digits()
X = digits.data
y = to_categorical(digits.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 200, num_classes, 'accuracy')
batch_size = 10
epochs = 20

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


64
Epoch 1/20
144/144 [==============================] - 2s 4ms/step - loss: 2.6331 - accuracy: 0.4099
Epoch 2/20
144/144 [==============================] - 1s 4ms/step - loss: 16.9314 - accuracy: 0.3354
Epoch 3/20
144/144 [==============================] - 1s 4ms/step - loss: 9.9765 - accuracy: 0.3793
Epoch 4/20
144/144 [==============================] - 1s 4ms/step - loss: 1.9988 - accuracy: 0.6006
Epoch 5/20
144/144 [==============================] - 1s 4ms/step - loss: 0.3387 - accuracy: 0.8135
Epoch 6/20
144/144 [==============================] - 1s 4ms/step - loss: 0.0856 - accuracy: 0.8970
Epoch 7/20
144/144 [==============================] - 1s 4ms/step - loss: 0.0582 - accuracy: 0.9116
Epoch 8/20
144/144 [==============================] - 1s 4ms/step - loss: 0.0466 - accuracy: 0.9339
Epoch 9/20
144/144 [==============================] - 1s 4ms/step - loss: 0.0404 - accuracy: 0.9388
Epoch 10/20
144/144 [==============================] - 1s 4ms/step - loss: 0.0373 - accuracy: 0.

In [19]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
wine = load_wine()
X = wine.data
y = to_categorical(wine.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, num_classes, 'accuracy')
batch_size = 10
epochs = 20

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")

13
Epoch 1/20
15/15 [==============================] - 2s 5ms/step - loss: 1.1348 - accuracy: 0.5070
Epoch 2/20
15/15 [==============================] - 0s 4ms/step - loss: 0.4252 - accuracy: 0.6268
Epoch 3/20
15/15 [==============================] - 0s 4ms/step - loss: 0.2434 - accuracy: 0.7676
Epoch 4/20
15/15 [==============================] - 0s 4ms/step - loss: 0.1747 - accuracy: 0.8521
Epoch 5/20
15/15 [==============================] - 0s 4ms/step - loss: 0.1334 - accuracy: 0.9085
Epoch 6/20
15/15 [==============================] - 0s 4ms/step - loss: 0.1120 - accuracy: 0.9296
Epoch 7/20
15/15 [==============================] - 0s 5ms/step - loss: 0.0944 - accuracy: 0.9366
Epoch 8/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0820 - accuracy: 0.9507
Epoch 9/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0716 - accuracy: 0.9648
Epoch 10/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0639 - accuracy: 0.9648
Epoch 11/20
15/1

In [23]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'accuracy')
batch_size = 10
epochs = 20

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


30
Epoch 1/20
46/46 [==============================] - 2s 4ms/step - loss: 1.3719 - accuracy: 0.6615
Epoch 2/20
46/46 [==============================] - 0s 4ms/step - loss: 0.5659 - accuracy: 0.7275
Epoch 3/20
46/46 [==============================] - 0s 4ms/step - loss: 0.6117 - accuracy: 0.7890
Epoch 4/20
46/46 [==============================] - 0s 4ms/step - loss: 0.3210 - accuracy: 0.8264
Epoch 5/20
46/46 [==============================] - 0s 4ms/step - loss: 0.1550 - accuracy: 0.8396
Epoch 6/20
46/46 [==============================] - 0s 4ms/step - loss: 0.1156 - accuracy: 0.8967
Epoch 7/20
46/46 [==============================] - 0s 4ms/step - loss: 0.1097 - accuracy: 0.9209
Epoch 8/20
46/46 [==============================] - 0s 4ms/step - loss: 0.1032 - accuracy: 0.9187
Epoch 9/20
46/46 [==============================] - 0s 4ms/step - loss: 0.0888 - accuracy: 0.9341
Epoch 10/20
46/46 [==============================] - 0s 4ms/step - loss: 0.0886 - accuracy: 0.9363
Epoch 11/20
46/4

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import timeit
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, concatenate, Activation
from keras import backend as K
from keras.utils import get_custom_objects
from tensorflow.keras.optimizers import Adam
from sklearn import datasets




# Load Boston Housing data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the model
input_dim = X_train.shape[1]
optimized_model = create_optimized_model(input_dim, input_dim , 1 , 'mae') # input_dim and hidden_dim
batch_size = 5
epochs = 100
start = timeit.default_timer()
history = optimized_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")


Epoch 1/100
81/81 [==============================] - 2s 7ms/step - loss: 422.4798 - mae: 18.4384 - val_loss: 285.5947 - val_mae: 15.0036
Epoch 2/100
81/81 [==============================] - 0s 5ms/step - loss: 189.0670 - mae: 11.4974 - val_loss: 136.1927 - val_mae: 10.0943
Epoch 3/100
81/81 [==============================] - 0s 5ms/step - loss: 74.8571 - mae: 6.9165 - val_loss: 78.5353 - val_mae: 6.8714
Epoch 4/100
81/81 [==============================] - 0s 5ms/step - loss: 38.5398 - mae: 4.6600 - val_loss: 59.3132 - val_mae: 5.6685
Epoch 5/100
81/81 [==============================] - 0s 5ms/step - loss: 30.3591 - mae: 4.2077 - val_loss: 57.0448 - val_mae: 5.5512
Epoch 6/100
81/81 [==============================] - 0s 5ms/step - loss: 26.6982 - mae: 3.9680 - val_loss: 48.9142 - val_mae: 5.0451
Epoch 7/100
81/81 [==============================] - 0s 5ms/step - loss: 23.9488 - mae: 3.7597 - val_loss: 43.0389 - val_mae: 4.6806
Epoch 8/100
81/81 [==============================] - 0s 5ms/s

In [30]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler


# Load the dataset
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features to be between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'mae')
batch_size = 10
epochs = 100

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


10
Epoch 1/100
36/36 [==============================] - 2s 4ms/step - loss: 28582.7051 - mae: 150.3803
Epoch 2/100
36/36 [==============================] - 0s 4ms/step - loss: 24822.7812 - mae: 137.9283
Epoch 3/100
36/36 [==============================] - 0s 5ms/step - loss: 14633.7646 - mae: 98.0223
Epoch 4/100
36/36 [==============================] - 0s 4ms/step - loss: 5654.5830 - mae: 59.5233
Epoch 5/100
36/36 [==============================] - 0s 4ms/step - loss: 5073.3301 - mae: 57.2022
Epoch 6/100
36/36 [==============================] - 0s 4ms/step - loss: 4738.1538 - mae: 55.6386
Epoch 7/100
36/36 [==============================] - 0s 4ms/step - loss: 4529.4351 - mae: 54.7821
Epoch 8/100
36/36 [==============================] - 0s 4ms/step - loss: 4381.6143 - mae: 54.2393
Epoch 9/100
36/36 [==============================] - 0s 4ms/step - loss: 4327.6309 - mae: 53.4523
Epoch 10/100
36/36 [==============================] - 0s 4ms/step - loss: 4067.7883 - mae: 51.9712
Epoch 11/10

In [31]:
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
faces = fetch_olivetti_faces()
X = faces.images
y = to_categorical(faces.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reshape((-1, 64*64))
X_test = X_test.reshape((-1, 64*64))

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, num_classes, 'accuracy')
batch_size = 10
epochs = 20

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data
4096
Epoch 1/20
32/32 [==============================] - 2s 4ms/step - loss: 1.7785 - accuracy: 0.0188
Epoch 2/20
32/32 [==============================] - 0s 4ms/step - loss: 0.5852 - accuracy: 0.0375
Epoch 3/20
32/32 [==============================] - 0s 4ms/step - loss: 0.3625 - accuracy: 0.0500
Epoch 4/20
32/32 [==============================] - 0s 4ms/step - loss: 0.1358 - accuracy: 0.0844
Epoch 5/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0617 - accuracy: 0.2250
Epoch 6/20
32/32 [==============================] - 0s 5ms/step - loss: 0.0447 - accuracy: 0.2688
Epoch 7/20
32/32 [==============================] - 0s 5ms/step - loss: 0.0362 - accuracy: 0.4437
Epoch 8/20
32/32 [==============================] - 0s 5ms/step - loss: 0.0310 - accuracy: 0.5125
Epoch 9/20
32/32 [==============================] - 0s 6ms/step - loss: 0.0275 - accuracy: 0.6531
Epoch 1

In [33]:
from sklearn.datasets import load_linnerud
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load and preprocess the dataset
linnerud = load_linnerud()
X = linnerud.data
y = linnerud.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'mae')
batch_size = 10
epochs = 100

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")

3
Epoch 1/100
2/2 [==============================] - 2s 13ms/step - loss: 12183.2539 - mae: 89.7331
Epoch 2/100
2/2 [==============================] - 0s 9ms/step - loss: 12163.3691 - mae: 89.6184
Epoch 3/100
2/2 [==============================] - 0s 9ms/step - loss: 12148.4424 - mae: 89.5347
Epoch 4/100
2/2 [==============================] - 0s 9ms/step - loss: 12133.5527 - mae: 89.4530
Epoch 5/100
2/2 [==============================] - 0s 9ms/step - loss: 12117.9482 - mae: 89.3636
Epoch 6/100
2/2 [==============================] - 0s 8ms/step - loss: 12099.7412 - mae: 89.2621
Epoch 7/100
2/2 [==============================] - 0s 9ms/step - loss: 12085.3242 - mae: 89.1800
Epoch 8/100
2/2 [==============================] - 0s 8ms/step - loss: 12070.1846 - mae: 89.0918
Epoch 9/100
2/2 [==============================] - 0s 8ms/step - loss: 12054.3633 - mae: 89.0009
Epoch 10/100
2/2 [==============================] - 0s 12ms/step - loss: 12036.9561 - mae: 88.9000
Epoch 11/100
2/2 [=======